In [1]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv
import datetime

In [2]:
total_articles = [] # it will save all publications

In [3]:
## get days
iterator_day = datetime.date(2019, 8, 14)
last_publication = datetime.date(2014, 1, 1)

In [4]:
array_days = [str(iterator_day)]
while(iterator_day >= last_publication):
    iterator_day = iterator_day - datetime.timedelta(days=1)
    array_days.append(str(iterator_day))

In [5]:
def save_csv(articles):
    with open('el_comercio_8_14.csv', 'wb') as f:
            dict_writer = csv.DictWriter(
                f,
                [
                    'Titulo',
                    'Resumen',
                    'Categoria', 
                    'Autor', 
                    'Fecha', 
                    'Contenido', 
                    'Etiquetas',
                    'Enlace'
                ],
                encoding='utf-8')
            dict_writer.writeheader()
            dict_writer.writerows(articles)

In [6]:
def get_title(content_soup):
    if content_soup.find('h1', {'class': 'news-title'}) is not None:
        return content_soup.find('h1', {'class': 'news-title'}).text
    
    return;

In [7]:
def get_summary(content_soup):
    if content_soup.find('h2', {'class': 'news-summary'}) is not None:
        return content_soup.find('h2', {'class': 'news-summary'}).text
    
    return;

In [8]:
def get_category(content_soup):
    if content_soup.find('h3', {'class': 'news-category fija'}) is not None:
        if content_soup.find('h3', {'class': 'news-category fija'}).find('a') is not None:
            return content_soup.find('h3', {'class': 'news-category fija'}).find('a').text
    
    return;

In [9]:
def get_author(content_soup):
    if content_soup.find('div', {'class': 'news-author-date'}) is not None:
        if content_soup.find('div', {'class': 'news-author-date'}).find('a') is not None:
            if content_soup.find('div', {'class': 'news-author-date'}).find('a').find('span') is not None:
                return content_soup.find('div', {'class': 'news-author-date'}).find('a').find('span').text
    
    return;

In [10]:
def get_date(content_soup):
    if content_soup.find('div', {'class': 'news-author-date'}) is not None:
        if content_soup.find('div', {'class': 'news-author-date'}).find('time') is not None:
            return content_soup.find('div', {'class': 'news-author-date'}).find('time')['datetime']
    
    return;

In [11]:
def get_content(content_soup):
    if content_soup.find('div', {'class': 'news-text-content'}) is not None:
        if len(content_soup.find('div', {'class': 'news-text-content'}).findAll('p', {'class': 'parrafo'})) > 0:
            return content_soup.find('div', {'class': 'news-text-content'}).findAll('p', {'class': 'parrafo'})
    
    return [];

In [12]:
def get_tags(content_soup):
    if len(content_soup.findAll('h2', {'class': 'tags-item'})) > 0:
        return content_soup.findAll('h2', {'class': 'tags-item'})
    
    return [];

In [13]:
def interate_articles_and_save(articles):
    cont = 0
    for article in articles:
        print('-------------------- Article ----------------------')
        url_article = 'https://elcomercio.pe' + article.find('a', {'class': 'page-link'})['href']
        content_article = requests.get(url_article).content
        soup_article = BeautifulSoup(content_article,'lxml')
        print("URL Article===>" + str(url_article))
        
        title = get_title(soup_article)
        summary = get_summary(soup_article)
        category = get_category(soup_article)
        author = get_author(soup_article)
        date = get_date(soup_article)
        text_content = get_content(soup_article)
        content_article = []
        for text in text_content:    
            content_article.append(text.text)

        tags_article = get_tags(soup_article)
        tags = []
        for tag in tags_article:
            tags.append(tag.text)

        total_articles.extend([
            {
                'Titulo':str(title), 
                'Resumen': str(summary), 
                'Categoria':str(category), 
                'Autor':str(author),
                'Fecha':str(date),
                'Contenido':str(content_article),
                'Etiquetas':str(tags),
                'Enlace':url_article
            }
        ])
        save_csv(total_articles)
        cont = cont + 1
        print('-----------------------------------  ' + str(cont*100/len(articles)) + "%")
        print('-------------------- End Article ----------------------')

In [14]:
def iterate_publications(days):
    for day in days:
        print('-------------------- Day ----------------------')
        url_day = 'https://elcomercio.pe/archivo/todas/' + day
        content_day = requests.get(url_day).content
        soup_day = BeautifulSoup(content_day,'lxml')
        print("URL Day===>" + str(url_day))
        
        articles_day = soup_day.findAll('article', {'class' : 'section-flow'})
        try:
            interate_articles_and_save(articles_day)
        except:
            print('Error para obtener pagina====> ' + url_day)
        print('-------------------- End Day ----------------------')

In [15]:
iterate_publications(array_days)

-------------------- Day ----------------------
URL Day===>https://elcomercio.pe/archivo/todas/2019-08-14
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/economia/mercados/precio-dolar-mexico-hoy-miercoles-14-agosto-2019-tipo-cambio-banamex-sat-peso-mexicano-noticia-665204
-----------------------------------  0.34965034965034963%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/somos/historias/pueblo-libre-tabernas-limenas-son-parte-nuestra-identidad-ciudad-ecpm-noticia-597615
-----------------------------------  0.6993006993006993%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/lima/sucesos/olivos-familia-busca-hombre-69-padece-alzheimer-noticia-665492
-----------------------------------  1.048951048951049%
-------------------- End Article --------------

URL Article===>https://elcomercio.pe/mundo/eeuu/tiroteo-filadelfia-hombre-le-dispara-policias-hay-heridos-noticia-665374
-----------------------------------  9.79020979020979%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/panamericanos-lima-2019/daniella-rosas-sofia-siempre-estuvo-detras-mi-mejorar-ecpm-noticia-664735
-----------------------------------  10.13986013986014%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/peru/apurimac/apurimac-vecinos-impiden-paso-vehiculos-via-evitamiento-fotos-noticia-665477
-----------------------------------  10.48951048951049%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/santiago-silva-dio-positivo-control-antidopaje-quedo-suspen

URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/liverpool-vs-chelsea-adrian-arquero-tenia-equipo-pieza-clave-supercopa-europa-liverpool-noticia-ecpm-664999
-----------------------------------  19.23076923076923%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/lima/callao/callao-jovenes-podran-acceder-puesto-semana-bellavista-noticia-nndc-665407
-----------------------------------  19.58041958041958%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tecnologia/tecnologia/han-prohibido-macbook-pro-vuelos-ee-uu-europa-mexico-espana-argentina-colombia-noticia-665148
-----------------------------------  19.93006993006993%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tecnologia/actualida

URL Article===>https://elcomercio.pe/luces/musica/shawn-mendes-camila-cabello-actuaran-mtv-video-music-awards-2019-noticia-nndc-665358
-----------------------------------  28.67132867132867%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/facebook-liverpool-vs-chelsea-vivo-ver-golazo-mane-dio-2-1-reds-suplementario-supercopa-europa-2019-2020-video-yt-youtube-viral-noticia-665351
-----------------------------------  29.02097902097902%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/liverpool-campeon-supercopa-europa-informo-prensa-internacional-fotos-noticia-nczd-665375
-----------------------------------  29.37062937062937%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Arti

URL Article===>https://elcomercio.pe/tvmas/famosos/kylie-jenner-travis-scott-realizan-increible-clavado-techo-yate-video-noticia-nndc-665323
-----------------------------------  38.11188811188811%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/deporte-total/futbol-mundial/liverpool-vs-chelsea-stephanie-frappart-anulo-segundo-gol-blues-video-noticia-665345
-----------------------------------  38.46153846153846%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/economia/mundo/alemania-empieza-golpeada-guerra-comercial-noticia-665328
-----------------------------------  38.81118881118881%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/luces/vida-social/horoscopo-hoy-miercoles-14-agosto-2019-gratis-amor-

...

-----------------------------------  66.55405405405405%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/luces/cine/harry-potter-trasciende-cumple-39-anos-fotos-video-noticia-nndc-660627
-----------------------------------  66.89189189189189%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/mundo/actualidad/ekaterina-karaglanova-misteriosa-muerte-estrella-rusa-instagram-cuyo-cuerpo-hallado-maleta-noticia-660561
-----------------------------------  67.22972972972973%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/economia/mercados/dolar-tipo-cambio-precio-dolar-peru-hoy-31-julio-2019-compra-venta-sbs-interbancario-ocona-noticia-nndc-660629
-----------------------------------  67.56756756756756%
-------

-----------------------------------  75.67567567567568%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/mundo/actualidad/pacto-estados-unidos-guatemala-perjudica-migrantes-honduras-salvador-donald-trump-bbc-noticia-660577
-----------------------------------  76.01351351351352%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/politica/juan-sheput-fuerza-popular-debe-dejar-presidencia-constitucion-noticia-nndc-660582
-----------------------------------  76.35135135135135%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tvmas/farandula/laura-bozzo-mexico-nombrada-doctora-honoris-causa-recibe-medalla-benito-juarez-fotos-noticia-nndc-660595
-----------------------------------  76.6891891891892%
-----------

-----------------------------------  84.79729729729729%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/politica/levitsky-vizcarra-necesita-dosis-populismo-enfrentar-fujimorismo-ecpm-noticia-660266
-----------------------------------  85.13513513513513%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/mundo/asia/youtube-sobrevivio-nino-3-anos-impresionante-caida-quinto-piso-china-noticia-nndc-660572
-----------------------------------  85.47297297297297%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/luces/cine/avengers-infinity-war-endgame-marvel-studios-revelo-escenas-falsas-formato-animado-fotos-video-noticia-nndc-660570
-----------------------------------  85.8108108108108%
-------------------- E

URL Article===>https://elcomercio.pe/peru/gobernadores-regionales-norte-oriente-respaldan-propuesta-adelanto-elecciones-noticia-660153
-----------------------------------  94.25675675675676%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tecnologia/ciencias/cancer-10-tratamientos-alternativos-recomendados-expertos-mexico-colombia-argentina-noticia-660308
-----------------------------------  94.5945945945946%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/tvmas/farandula/instagram-luis-fonsi-paso-peru-fotos-cantante-compartio-redes-noticia-660297
-----------------------------------  94.93243243243244%
-------------------- End Article ----------------------
-------------------- Article ----------------------
URL Article===>https://elcomercio.pe/politica/araoz-vizcarra-gran-parada-militar-creo-ha

KeyboardInterrupt: 